In [1]:
!pip install google-play-scraper

  Using cached google_play_scraper-1.2.7-py3-none-any.whl.metadata (50 kB)
Using cached google_play_scraper-1.2.7-py3-none-any.whl (28 kB)



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
!pip install pandas

  Using cached numpy-2.2.6-cp313-cp313-win_amd64.whl.metadata (60 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
    --------------------------------------- 0.3/11.0 MB ? eta -:--:--
   ------ --------------------------------- 1.8/11.0 MB 7.6 MB/s eta 0:00:02
   --------- ------------------------------ 2.6/11.0 MB 8.8 MB/s eta 0:00:01
   ------------ --------------------------- 3.4/11.0 MB 5.1 MB/s eta 0:00:02
   --------------- ------------------------ 4.2/11.0 MB 4.6 MB/s eta 0:00:02
   ----------------- ---------------------- 4.7/11.0 MB 4.3 MB/s eta 0:00:02
   ------------------- -------------------- 5.2/11.0 MB 3.9 MB/s eta 0:00:02
   ------------------------ --------------- 6.8/11.0 MB 4.4 MB/s eta 0:00:01
   ------------------------------- -------- 8.7/11.0 MB 4.8 MB/s eta 0:00:01
   ---------------------------------- ---


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
from google_play_scraper import reviews, Sort
import pandas as pd
from datetime import datetime

# Define bank apps with their correct app IDs

In [5]:
# Step 3: Define bank apps with their correct app IDs
banks = [
    {'app_id': 'com.combanketh.mobilebanking', 'name': 'Commercial Bank of Ethiopia'},
    {'app_id': 'com.boa.boaMobileBanking', 'name': 'Bank of Abyssinia'},
    {'app_id': 'com.cr2.amolelight', 'name': 'Dashen Bank'}
]


In [6]:
def get_reviews_for_bank(app_id, bank_name, count=1000):
    all_reviews = []
    batch_size = 200
    
    for _ in range(0, count, batch_size):
        revs, _ = reviews(
            app_id,
            lang=None,        # all languages
            sort=Sort.NEWEST,
            count=batch_size,
            filter_score_with=None
        )
        
        for r in revs:
            all_reviews.append({
                'review': r['content'],
                'rating': r['score'],
                'date': r['at'].strftime('%Y-%m-%d'),
                'bank': bank_name,
                'source': 'Google Play'
            })
        
        if len(revs) < batch_size:
            break
    
    return all_reviews


# Collect reviews for all banks

In [7]:
# Step 5: Collect reviews for all banks
all_reviews = []

for bank in banks:
    print(f"Scraping reviews for {bank['name']}...")
    reviews_data = get_reviews_for_bank(bank['app_id'], bank['name'], count=400)
    print(f"Collected {len(reviews_data)} reviews for {bank['name']}")
    all_reviews.extend(reviews_data)


Scraping reviews for Commercial Bank of Ethiopia...
Collected 400 reviews for Commercial Bank of Ethiopia
Scraping reviews for Bank of Abyssinia...
Collected 400 reviews for Bank of Abyssinia
Scraping reviews for Dashen Bank...
Collected 400 reviews for Dashen Bank


In [8]:
# Step 6: Create DataFrame and clean data
df = pd.DataFrame(all_reviews)

# Remove duplicate reviews
df.drop_duplicates(subset=['review'], inplace=True)

# Drop missing values
df.dropna(inplace=True)

print(f"Total reviews after cleaning: {len(df)}")


Total reviews after cleaning: 485


# To remove the Emoji reviwes 

In [9]:
import re

def is_text_only_emoji(text):
    # Remove whitespace
    text = text.strip()
    # Return True if there are no word characters (like a-z, A-Z, 0-9)
    return not re.search(r'\w', text)

# Remove reviews with only emojis
df = df[~df['review'].apply(is_text_only_emoji)]


# Save cleaned data to CSV

In [10]:
df.to_csv('bank_reviews.csv', index=False, encoding='utf-8-sig')
